# WDB - Web scraping with selenium

Rami Tarabishi - @r9119

This notebook covers my attempt at webscraping company finiancials from [nasdaq.com](https://www.nasdaq.com/market-activity/stocks/screener)

I did make use of chatGPT and Googles Bard when I had errors and will mention my troubles under each cell where they were, but honestly most of the time the solutions didnt help and I just figured it out on my own.

In [1]:
# import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.DataFrame(columns=['company_name', 'ticker_symbol', 'market_cap', 'url'])
# # Init dataframe with the columns I want that are located in the main table
# # I am first only scraping the table with the ticker symbols as the financial data is located in link/ticker_symbol
# # to later iterate over the list of symbols and scrape the financial data seperately 
# df.head()
# df = pd.read_csv('../data/nasdaq_stock_list.csv')

df.head()

,company_name,ticker_symbol,market_cap,url
0,Apple Inc. Common Stock,AAPL,"3,043,829,093,920",https://www.nasdaq.com/market-activity/stocks/...
1,Microsoft Corporation Common Stock,MSFT,"2,781,375,531,382",https://www.nasdaq.com/market-activity/stocks/...
2,Alphabet Inc. Class C Capital Stock,GOOG,"1,710,186,240,000",https://www.nasdaq.com/market-activity/stocks/...
3,Alphabet Inc. Class A Common Stock,GOOGL,"1,689,534,840,000",https://www.nasdaq.com/market-activity/stocks/...
4,"Amazon.com, Inc. Common Stock",AMZN,"1,523,442,788,988",https://www.nasdaq.com/market-activity/stocks/...


In [3]:
chrome_options = Options()

# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-images")

In [4]:
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://www.nasdaq.com/market-activity/stocks/screener")
wait = WebDriverWait(driver, timeout=10)

In [5]:
# Close cookie consent banner as it blocks the pagination buttons
cookie_consent_exit_button = driver.find_element(By.XPATH, '//*[@id="onetrust-close-btn-container"]/button')
cookie_consent_exit_button.click()

In [6]:
def scrape_table_page(table_page):
    '''
    Function to parse and extract data from a table page
    -----
    Input:
        table_page: The page to be scraped
    -----
    Returns:
        df: A dataframe containing the extracted data from that page
    '''

    try:
        rows = table_page.find_elements(by=By.CLASS_NAME, value="nasdaq-screener__row")
    except:
        print("Error: No rows found")
        return None
    
    data_list = []
    for row in rows[1:]:
        try:
            ticker_symbol = row.find_element(by=By.CLASS_NAME, value="nasdaq-screener__cell")
            name = row.find_element(by=By.CLASS_NAME, value="nasdaq-screener__cell--name").text
            mc = row.find_element(by=By.CLASS_NAME, value="nasdaq-screener__cell--marketCap").text

            data_dict = {'company_name': name, 'ticker_symbol': ticker_symbol.text, 'market_cap': mc, 'url': ticker_symbol.find_element(by=By.TAG_NAME, value="a").get_attribute('href')}
            data_list.append(data_dict)
        except TimeoutException:
            print("Error: Couldnt extract data from row")
            continue
    
    return pd.DataFrame.from_dict(data_list)

In [7]:
# First page of table
stock_table = driver.find_element(by=By.CLASS_NAME, value="nasdaq-screener__table")
data_list = scrape_table_page(stock_table)
df = pd.concat([df, data_list], ignore_index=True)

# # Iterate over pages
while True:
    try:
        next_page_button = driver.find_element(by=By.CLASS_NAME, value="pagination__next")
    except TimeoutException:
        raise Exception("Error: Couldnt find next page button")
    
    if next_page_button.get_attribute("disabled") == "true":
        print("Reached last page")
        break
    else:
        try:
            # Force scroll to button and click (Button could be hidden by a banner)
            driver.execute_script(f'window.scroll(0, {next_page_button.location["y"]});')
            next_page_button.click()
            try:
                # Wait for the new table to load
                # time.sleep(5) # Dynamic waiting wasnt working so here we are (I figured it out ↓)
                div_location = (By.CSS_SELECTOR, ".nasdaq-screener__data.loaded")
                wait.until(EC.visibility_of_element_located(div_location))

                # Extract new table page
                stock_table = driver.find_element(by=By.CLASS_NAME, value="nasdaq-screener__table")

                # Parse and extract data from new table page
                data_list = scrape_table_page(stock_table)
                df = pd.concat([df, data_list], ignore_index=True)
            except TimeoutException:
                raise Exception("Error: Couldnt find new table page")
        except TimeoutException:
            raise Exception("Error: Couldnt click next page button")

Reached last page


Like the comments say, I had some issues with the waiting between page loads, neither gpt3.5, 4 or bard helped much so I just used a time.sleep() function at first, and after reading more of the docs and more help from bard I eventually understood how to properly use the wait functions. 

In [8]:
df.head()

,company_name,ticker_symbol,market_cap,url
0,Apple Inc. Common Stock,AAPL,"3,043,829,093,920",https://www.nasdaq.com/market-activity/stocks/...
1,Microsoft Corporation Common Stock,MSFT,"2,781,375,531,382",https://www.nasdaq.com/market-activity/stocks/...
2,Alphabet Inc. Class C Capital Stock,GOOG,"1,710,186,240,000",https://www.nasdaq.com/market-activity/stocks/...
3,Alphabet Inc. Class A Common Stock,GOOGL,"1,689,534,840,000",https://www.nasdaq.com/market-activity/stocks/...
4,"Amazon.com, Inc. Common Stock",AMZN,"1,523,442,788,988",https://www.nasdaq.com/market-activity/stocks/...


In [9]:
# Save stock list to data dir
# df.to_csv('../data/nasdaq_stock_list.csv', index=False)

In [10]:
financials_df = pd.DataFrame(columns=['ticker_symbol', 'total_revenue', 'gross_profit', 'net_income', 'total_assets', 'total_liabilities', 'year'])
# Init dataframe with the columns I want that are located in the financials table
financials_df

,ticker_symbol,total_revenue,gross_profit,net_income,total_assets,total_liabilities,year


In [11]:
def scrape_financials(ticker, table, financials_to_scrape):
    '''
    Function to parse and extract financial data from a table
    -----
    Input:
        ticker: str - ticker symbol
        table: selenium table element
        financials_to_scrape: list - list of strings of financials to scrape (e.g. ['Total Revenue', 'Gross Profit'])
    -----
    Returns:
        df: A pandas dataframe containing the extracted data from that table corresponding to financials_to_scrape
    '''
    # Get table rows
    try:
        rows = table.find_elements(by=By.TAG_NAME, value="tr")
    except:
        print("Error: No rows found")
        return None
    
    # Extract years from the table header
    years = []
    # Excluding the first column as its just the display name for recorded periods
    for row in rows[0].find_elements(by=By.TAG_NAME, value="th")[1:]:
        years.append({'ticker_symbol': ticker, 'year': row.text[-4:]})

    # Extract financial data from the table (First row is the header so we skip it)
    for row in rows[1:]:
        try:
            # Extract columns from each row
            cells = row.find_elements(by=By.CLASS_NAME, value="financials__cell")
            # Check if the row contains the data we want (financials_to_scrape)
            if cells[0].text in financials_to_scrape:
                for i, cell in enumerate(cells[1:]):
                    years[i][cells[0].text.lower().replace(' ', '_')] = cell.text
        except TimeoutException:
            print("Error: Couldnt extract data from row")
            continue

    return pd.DataFrame.from_dict(years)

In [12]:
def scrape_financials_deprec(ticker, incomeStatementTable, balanceSheetTable):
    '''
    Deprecated function to parse and extract financial data from a table
    Deprecated because I needed to scrape the data seperately as the tables technically loaded simultaneously, but only visible data could be scraped by selenium
    '''
    try:
        is_rows = incomeStatementTable.find_elements(by=By.TAG_NAME, value="tr")
        bs_rows = balanceSheetTable.find_elements(by=By.TAG_NAME, value="tr")
    except:
        print("Error: No rows found")
        return None
    
    years = []
    for row in is_rows[0].find_elements(by=By.TAG_NAME, value="th")[1:]:
        years.append({'ticker_symbol': ticker, 'year': row.text[-4:]})

    for row in is_rows[1:]:
        try:
            cells = row.find_elements(by=By.CLASS_NAME, value="financials__cell")
            # print(cells[0].text)
            if cells[0].text in ['Net Income', 'Total Revenue', 'Gross Profit']:
                # print(cells)
                for i, cell in enumerate(cells[1:]):
                    years[i][cells[0].text.lower().replace(' ', '_')] = cell.text
        except TimeoutException:
            print("Error: Couldnt extract data from row")
            continue

    for row in bs_rows[1:]:
        try:
            cells = row.find_elements(by=By.CLASS_NAME, value="financials__cell")
            print(cells[0].text, cells[1].text, cells[2].text, cells[3].text, cells[4].text)
            if cells[0].text in ['Total Assets', 'Total Liabilities']:
                for i, cell in enumerate(cells[1:]):
                    years[i][cells[0].text.lower().replace(' ', '_')] = cell.text
        except TimeoutException:
            print("Error: Couldnt extract data from row")
            continue

    return pd.DataFrame.from_dict(years)

In [13]:
reload_after_iters = 500

# Extracing the financial data for each ticker
for i, (ticker, url) in enumerate(zip(df['ticker_symbol'], df['url'])):
    try:
        driver.get(url + '/financials')
    except:
        raise Exception(f'Could not load {url + "/financials"}')
    
    # Wait until the data table we want is visible on the page:
    try:
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'financials__body')))

        # Extract the income statement and scrape the data
        try:
            is_table = driver.find_element(by=By.XPATH, value='//div[@data-panel-name="incomeStatementTable"]')
            is_data_list = scrape_financials(ticker, is_table, ['Total Revenue', 'Gross Profit', 'Net Income']) # this can be modified to scrape more or less data from the table
        except TimeoutException:                                                                                                 # just insure the strings match the table data
            print("Couldnt get income statement table")

        # Click onto balance sheet tab and scrape the data
        try:
            bs_button = driver.find_element(by=By.XPATH, value='//button[@data-value="balanceSheetTable"]')
            bs_button.click()
        except TimeoutException:
            print("Couldnt click onto balance sheet")

        try:
            bs_table = driver.find_element(by=By.XPATH, value='//div[@data-panel-name="balanceSheetTable"]')
            bs_data_list = scrape_financials(ticker, bs_table, ['Total Assets', 'Total Liabilities'])
        except TimeoutException:
            print("Couldnt get balance sheet table")

        # Merge the two dataframes
        data_list = pd.merge(is_data_list, bs_data_list, on=['ticker_symbol', 'year'])
        financials_df = pd.concat([financials_df, data_list], ignore_index=True)
    except TimeoutException:
        # Appending an empty dict with the ticker and year to the dataframe to indicate that no data was found
        no_data_dict = {'ticker_symbol': ticker, 'year': 'No data found'}
        temp_df = pd.DataFrame.from_dict([no_data_dict])
        financials_df = pd.concat([financials_df, temp_df], ignore_index=True)
        print(f'Could not find data tables on {url + "/financials"}')

    # Reload the page after every 500 iterations to prevent the page from crashing
    if i % reload_after_iters == 0:
        driver.quit()
        driver = webdriver.Chrome(options=chrome_options)
        wait = WebDriverWait(driver, timeout=10)
        continue

# for ticker, url in zip(df['ticker_symbol'], df['url']):
#     try:
#         driver.get(url + '/financials')
#     except:
#         raise Exception(f'Could not load {url + "/financials"}')
    
#     # Wait until the data table we want is visible on the page:
#     try:
#         wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'financials__body')))

#         # Extract the income statement and scrape the data
#         is_table = driver.find_element(by=By.XPATH, value='//div[@data-panel-name="incomeStatementTable"]')
#         is_data_list = scrape_financials(ticker, is_table, ['Total Revenue', 'Gross Profit', 'Net Income']) # this can be modified to scrape more or less data from the table
#                                                                                                             # just insure the strings match the table data
#         # Click onto balance sheet tab and scrape the data
#         bs_button = driver.find_element(by=By.XPATH, value='//button[@data-value="balanceSheetTable"]')
#         bs_button.click()

#         bs_table = driver.find_element(by=By.XPATH, value='//div[@data-panel-name="balanceSheetTable"]')
#         bs_data_list = scrape_financials(ticker, bs_table, ['Total Assets', 'Total Liabilities'])

#         # Merge the two dataframes
#         data_list = pd.merge(is_data_list, bs_data_list, on=['ticker_symbol', 'year'])
#         financials_df = pd.concat([financials_df, data_list], ignore_index=True)
#     except TimeoutException:
#         # Appending an empty dict with the ticker and year to the dataframe to indicate that no data was found
#         no_data_dict = {'ticker_symbol': ticker, 'year': 'No data found'}
#         temp_df = pd.DataFrame.from_dict([no_data_dict])
#         financials_df = pd.concat([financials_df, temp_df], ignore_index=True)
#         print(f'Could not find data tables on {url + "/financials"}')

Could not find data tables on https://www.nasdaq.com/market-activity/stocks/brk/a/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/brk/b/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/arm/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/sgen/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/kvue/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/bnh/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/bnj/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/cboe/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/vlto/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/bg/financials
Could not find data tables on https://www.nasdaq.com/market-activity/stocks/tko/fin

Exception: Could not load https://www.nasdaq.com/market-activity/stocks/tbnk/financials

In [14]:
financials_df.head()

,ticker_symbol,total_revenue,gross_profit,net_income,total_assets,total_liabilities,year
0,AAPL,"$383,285,000","$169,148,000","$96,995,000","$352,583,000","$290,437,000",2023
1,AAPL,"$394,328,000","$170,782,000","$99,803,000","$352,755,000","$302,083,000",2022
2,AAPL,"$365,817,000","$152,836,000","$94,680,000","$351,002,000","$287,912,000",2021
3,AAPL,"$274,515,000","$104,956,000","$57,411,000","$323,888,000","$258,549,000",2020
4,MSFT,"$211,915,000","$146,052,000","$72,361,000","$411,976,000","$205,753,000",2023


In [15]:
financials_df

,ticker_symbol,total_revenue,gross_profit,net_income,total_assets,total_liabilities,year
0,AAPL,"$383,285,000","$169,148,000","$96,995,000","$352,583,000","$290,437,000",2023
1,AAPL,"$394,328,000","$170,782,000","$99,803,000","$352,755,000","$302,083,000",2022
2,AAPL,"$365,817,000","$152,836,000","$94,680,000","$351,002,000","$287,912,000",2021
3,AAPL,"$274,515,000","$104,956,000","$57,411,000","$323,888,000","$258,549,000",2020
4,MSFT,"$211,915,000","$146,052,000","$72,361,000","$411,976,000","$205,753,000",2023
...,...,...,...,...,...,...,...
17991,SLNG,"$98,823","$21,129","-$3,186","$96,580","$36,713",2022
17992,SLNG,"$69,171","$13,955","-$7,798","$87,336","$26,277",2021
17993,SLNG,"$41,550","$11,539","-$6,756","$80,313","$18,283",2020
17994,SLNG,"$47,071","$15,200","-$5,715","$90,326","$22,483",2019


In [16]:
financials_df.to_csv('../data/nasdaq_listed_company_financials.csv', index=False)

In [17]:
driver.quit()